In [11]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

In [12]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [13]:
def get_all_imgs(url):
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            continue  # skip if img tag contains no src attr
        img_url = urljoin(url, img_url)
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [14]:
def download(url, pathname):
    """
    Download the file with the given URL, 
    and puts it in the folder followed by the pathname
    """
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
        
    # Download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    
    # Get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    
    # Get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    
    # Progress bar, changing the unit to bytes instead of iteration,
    # Which is default by tqdm
    progress = tqdm(response.iter_content(1024), \
                    f"Downloading{filename}", \
                    total=file_size, \
                    unit="B", \
                    unit_scale=True, \
                    unit_divisor=1024)
    
    with open(filename, "wb") as f:
        for data in progress.iterable:
            f.write(data)
            progress.update(len(data))

In [15]:
def main(url, path):
    imgs = get_all_imgs(url)
    for img in imgs:
        download(img, path)

In [20]:
main("http://www.beautyleg.com/photo/show.php?no=118", "test")


Extracting images: 100%|██████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 52966.74it/s]

Downloadingtest/BEAUTYLEG.gif: 100%|████████████████████████████████████████████████████████| 7.78k/7.78k [00:00<00:00, 4.82MB/s]

Downloadingtest/0000.jpg:   0%|                                                                      | 0.00/92.3k [00:00<?, ?B/s]
Downloadingtest/0000.jpg:  14%|████████▌                                                    | 13.0k/92.3k [00:00<00:01, 77.7kB/s]
Downloadingtest/0000.jpg:  39%|████████████████████████▏                                     | 36.0k/92.3k [00:00<00:00, 147kB/s]
Downloadingtest/0000.jpg:  57%|███████████████████████████████████▌                          | 53.0k/92.3k [00:00<00:00, 138kB/s]
Downloadingtest/0000.jpg: 100%|██████████████████████████████████████████████████████████████| 92.3k/92.3k [00:00<00:00, 169kB/s]

Downloadingtest/0001.jpg:   0%|                                                       

Downloadingtest/0016.jpg:   0%|                                                                      | 0.00/43.0k [00:00<?, ?B/s]
Downloadingtest/0016.jpg:  30%|██████████████████▍                                          | 13.0k/43.0k [00:00<00:00, 76.6kB/s]
Downloadingtest/0016.jpg: 100%|██████████████████████████████████████████████████████████████| 43.0k/43.0k [00:00<00:00, 121kB/s]

Downloadingtest/0017.jpg:   0%|                                                                      | 0.00/41.3k [00:00<?, ?B/s]
Downloadingtest/0017.jpg:  34%|████████████████████▋                                        | 14.0k/41.3k [00:00<00:00, 83.9kB/s]
Downloadingtest/0017.jpg: 100%|██████████████████████████████████████████████████████████████| 41.3k/41.3k [00:00<00:00, 119kB/s]

Downloadingtest/0018.jpg:   0%|                                                                      | 0.00/43.0k [00:00<?, ?B/s]
Downloadingtest/0018.jpg:  30%|██████████████████▍                                      

In [ ]:
http://www.beautyleg.com/photo/show.php?no=118

In [ ]:
http://photo.beautyleg.com/album/118-GZq956/0001.jpg

In [ ]:
http://photo.beautyleg.com/album/118-GZq956/0001.jpg